<a href="https://colab.research.google.com/github/jla524/f1-analysis/blob/master/predict_with_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [0]:
stock = 'AAPL'

In [0]:
pathname = str.lower(stock) + '.us.txt'
df = pd.read_csv(pathname)
cl = df.Close
df.tail()

In [0]:
plt.figure()
plt.plot(df["Close"])
plt.title(stock + ' stock price history')
plt.ylabel('Price (USD)')
plt.xlabel('Days')
plt.show()

In [0]:
plt.figure()
plt.plot(df["Volume"])
plt.title(stock + ' stock volume history')
plt.ylabel('Volume')
plt.xlabel('Days')
plt.show()

In [0]:
scl = MinMaxScaler()

In [0]:
cl = np.array(cl).reshape(-1, 1)
cl = scl.fit_transform(cl)

In [0]:
#Create a function to process the data into 7 day look back slices
def processData(data,lb):
    X,Y = [],[]
    for i in range(len(data)-lb-1):
        X.append(data[i:(i+lb),0])
        Y.append(data[(i+lb),0])
    return np.array(X),np.array(Y)
X,y = processData(cl,7)
X_train,X_test = X[:int(X.shape[0]*0.80)],X[int(X.shape[0]*0.80):]
y_train,y_test = y[:int(y.shape[0]*0.80)],y[int(y.shape[0]*0.80):]

In [0]:
#Build the model
model = Sequential()
model.add(LSTM(256,input_shape=(7,1)))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse')

In [0]:
#Reshape data for (Sample,Timestep,Features) 
X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],1))
X_test = X_test.reshape((X_test.shape[0],X_test.shape[1],1))

In [0]:
#Fit model with history to check for overfitting
history = model.fit(X_train,y_train,epochs=150,validation_data=(X_test,y_test),shuffle=False)

In [0]:
Xt = model.predict(X_test)
plt.plot(scl.inverse_transform(y_test.reshape(-1,1)))
plt.plot(scl.inverse_transform(Xt))